# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [53]:
import pandas as pd
import numpy as np
import requests
import json
import time
import folium

In [2]:
term = 'Pizza'
location = 'Williamsburg NY'
SEARCH_LIMIT = 10

api_key = 'f-e0nkjzRKGK7Xc_NSo8bwnSMwMwgs0O1nwWuKUhAcM3eRnLc61uOTuG1Y5oNAno_ljtqghJp6BgAFH_wy3dA-FjHaXbpkc5ORGpNuWBA1GKWRsKkYfqJeAdllyRXHYx'

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT}

response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])



<Response [200]>
<class 'str'>
{"businesses": [{"id": "v1DHGRNCH9247WLYoaoA9A", "alias": "l-industrie-pizzeria-brooklyn", "name": "L'industrie Pizzeria", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/SRL_RztRRUijgQwt67daGw/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/l-industrie-pizzeria-brooklyn?adjust_creative=E4PPiS_1lUpU1r53MEtBCQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=E4PPiS_1lUpU1r53MEtBCQ", "review_count": 401, "categories": [{"alias": "pizza", "title": "Pizza"}], "rating": 5.0, "coordinates": {"latitude": 40.71162, "longitude": -73.95783}, "transactions": ["delivery", "pickup"], "price": "$", "location": {"address1": "254 S 2nd St", "address2": "", "address3": "", "city": "Brooklyn", "zip_code": "11211", "country": "US", "state": "NY", "display_address": ["254 S 2nd St", "Brooklyn, NY 11211"]}, "phone": "+17185990002", "display_phone": "(718) 599-0002", "distance": 173.64614967576486}, {"id": "GRXvvB1OKMuW4XDWEaVAlg", "alia

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [3]:
# Your code here; use a function or loop to retrieve all the results from your original request
def api_request(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)

    df = pd.DataFrame(response.json()['businesses'])
    return df


In [21]:
def get_all_results(url_params,api_key):
    num_of_results = response.json()['total']
    print('{} total results'.format(num_of_results))
    current = 0
    dfs = []
    while current < num_of_results and current < 1000:
        url_params['offset'] = current
        dfs.append(api_request(url_params, api_key))
        time.sleep(1) #tells script to wait 1 second before calling again
        current += 50
    df = pd.concat(dfs, ignore_index=True)
    return df

In [23]:
# df = api_request(url_params, api_key)
df = get_all_results(url_params,api_key)

378 total results


In [24]:
df

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,l-industrie-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71162, 'longitude': -73.95783}",(718) 599-0002,173.646150,v1DHGRNCH9247WLYoaoA9A,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,"{'address1': '254 S 2nd St', 'address2': '', '...",L'industrie Pizzeria,+17185990002,$,5.0,401,"[delivery, pickup]",https://www.yelp.com/biz/l-industrie-pizzeria-...
1,joes-pizza-brooklyn-8,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71691, 'longitude': -73.958987}",(718) 388-2216,635.514917,GRXvvB1OKMuW4XDWEaVAlg,https://s3-media3.fl.yelpcdn.com/bphoto/zFqYcj...,False,"{'address1': '216 Bedford Ave', 'address2': ''...",Joe's Pizza,+17183882216,$,4.0,237,"[delivery, pickup]",https://www.yelp.com/biz/joes-pizza-brooklyn-8...
2,emmy-squared-brooklyn-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.712166, 'longitude': -73.955705}",(718) 360-4535,367.178059,UuY1QCxWybMF0IMxAYG06w,https://s3-media2.fl.yelpcdn.com/bphoto/ckUwbE...,False,"{'address1': '364 Grand St', 'address2': '', '...",Emmy Squared - Brooklyn,+17183604535,$$,4.0,529,[],https://www.yelp.com/biz/emmy-squared-brooklyn...
3,motorino-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.710559, 'longitude': -73.963462}",(718) 599-8899,302.281310,vLaaNR3rTeSFEZe_nn43BA,https://s3-media3.fl.yelpcdn.com/bphoto/AkVsxl...,False,"{'address1': '139 Broadway', 'address2': '', '...",Motorino,+17185998899,$$,4.0,222,"[delivery, pickup]",https://www.yelp.com/biz/motorino-brooklyn-2?a...
4,vinnies-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.718925, 'longitude': -73.956878}",(718) 782-7078,890.174079,lxPVSGnZ7YTiO5OGrqBS8Q,https://s3-media2.fl.yelpcdn.com/bphoto/T35Yd0...,False,"{'address1': '148 Bedford Ave', 'address2': ''...",Vinnie's Pizzeria,+17187827078,$,4.0,612,"[delivery, pickup]",https://www.yelp.com/biz/vinnies-pizzeria-broo...
5,roebling-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7084890714263, 'longitude': -7...",(718) 782-5042,320.853641,aTmNnrx0XE_nTSFFzXnt8w,https://s3-media1.fl.yelpcdn.com/bphoto/RFgh8r...,False,"{'address1': '326 Roebling St', 'address2': ''...",Roebling Pizza,+17187825042,$,4.0,103,"[delivery, pickup]",https://www.yelp.com/biz/roebling-pizza-brookl...
6,rosas-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.7139499, 'longitude': -73.95563}",(718) 963-2378,463.584840,aPn2_EtjfHduXEmEBNxuFA,https://s3-media3.fl.yelpcdn.com/bphoto/Ztmmhj...,False,"{'address1': '374 Metropolitan Ave', 'address2...",Rosa's Pizza,+17189632378,NaN,4.5,37,"[delivery, pickup]",https://www.yelp.com/biz/rosas-pizza-brooklyn?...
7,best-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7156, 'longitude': -73.95352}",(718) 599-2210,725.041014,G4KNXsc7F49OzkW7I28JCg,https://s3-media3.fl.yelpcdn.com/bphoto/nk5nX-...,False,"{'address1': '33 Havemeyer St', 'address2': ''...",Best Pizza,+17185992210,$,3.5,525,[delivery],https://www.yelp.com/biz/best-pizza-brooklyn?a...
8,anna-maria-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7179537220535, 'longitude': -7...",(718) 599-4550,775.870437,d0ic0tnQjujPsjQNuC1awQ,https://s3-media4.fl.yelpcdn.com/bphoto/Dnzvby...,False,"{'address1': '179 Bedford Ave', 'address2': ''...",Anna Maria Pizza,+17185994550,$,4.0,338,"[delivery, pickup]",https://www.yelp.com/biz/anna-maria-pizza-broo...
9,paulie-gees-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.729546, 'longitude': -73.958568}",(347) 987-3747,2034.606009,pGr5QuQU7Ur5GHLvXbT8bw,https://s3-media2.fl.yelpcdn.com/bphoto/1j39J4...,False,"{'address1': '60 Greenpoint Ave', 'address2': ...",Paulie Gee's,+13479873747,$$,4.5,1154,[],https://www.yelp.com/biz/paulie-gees-brooklyn?...


In [26]:
df.describe()

,distance,rating,review_count
count,80.000000,80.00000,80.00000
mean,1620.301890,3.70625,308.66250
std,884.715625,0.59371,418.32517
min,105.310949,1.50000,2.00000
25%,771.311044,3.50000,68.50000
50%,1606.833640,4.00000,158.00000
75%,2522.217969,4.00000,401.25000
max,2938.244844,5.00000,2125.00000


In [22]:
df.columns

Index(['alias', 'categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')

## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [10]:
df['rating'].mean()

4.15

In [31]:
df['price'].value_counts()

$$      39
$       28
$$$      5
$$$$     1
Name: price, dtype: int64

In [48]:
cheap_good_pizza = df.loc[(df['price'] == '$') & (df['rating'] >= 4) & (df['review_count'] > 100)]
cheap_good_pizza.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,l-industrie-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71162, 'longitude': -73.95783}",(718) 599-0002,173.646150,v1DHGRNCH9247WLYoaoA9A,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,"{'address1': '254 S 2nd St', 'address2': '', '...",L'industrie Pizzeria,+17185990002,$,5.0,401,"[delivery, pickup]",https://www.yelp.com/biz/l-industrie-pizzeria-...
1,joes-pizza-brooklyn-8,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71691, 'longitude': -73.958987}",(718) 388-2216,635.514917,GRXvvB1OKMuW4XDWEaVAlg,https://s3-media3.fl.yelpcdn.com/bphoto/zFqYcj...,False,"{'address1': '216 Bedford Ave', 'address2': ''...",Joe's Pizza,+17183882216,$,4.0,237,"[delivery, pickup]",https://www.yelp.com/biz/joes-pizza-brooklyn-8...
4,vinnies-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.718925, 'longitude': -73.956878}",(718) 782-7078,890.174079,lxPVSGnZ7YTiO5OGrqBS8Q,https://s3-media2.fl.yelpcdn.com/bphoto/T35Yd0...,False,"{'address1': '148 Bedford Ave', 'address2': ''...",Vinnie's Pizzeria,+17187827078,$,4.0,612,"[delivery, pickup]",https://www.yelp.com/biz/vinnies-pizzeria-broo...
5,roebling-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7084890714263, 'longitude': -7...",(718) 782-5042,320.853641,aTmNnrx0XE_nTSFFzXnt8w,https://s3-media1.fl.yelpcdn.com/bphoto/RFgh8r...,False,"{'address1': '326 Roebling St', 'address2': ''...",Roebling Pizza,+17187825042,$,4.0,103,"[delivery, pickup]",https://www.yelp.com/biz/roebling-pizza-brookl...
8,anna-maria-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7179537220535, 'longitude': -7...",(718) 599-4550,775.870437,d0ic0tnQjujPsjQNuC1awQ,https://s3-media4.fl.yelpcdn.com/bphoto/Dnzvby...,False,"{'address1': '179 Bedford Ave', 'address2': ''...",Anna Maria Pizza,+17185994550,$,4.0,338,"[delivery, pickup]",https://www.yelp.com/biz/anna-maria-pizza-broo...


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [50]:
lat = 40.6782
long = -73.949997

In [52]:
base_map = folium.Map([lat,long], zoom_start = 13)
x = cheap_good_pizza[]

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!